In [4]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(os.getenv("DATABASE_URL"))

# 1) Determine observation end aligned with events
obs_sql = """
SELECT LEAST(
    (SELECT MAX(feature_date) FROM hpce.customer_features_daily),
    (SELECT MAX(event_time::date) FROM hpce.rees46_events)
) AS obs_end;
"""
obs_end = pd.read_sql(obs_sql, engine)["obs_end"].iloc[0]
obs_end = pd.to_datetime(obs_end)

# 2) Choose 6 monthly snapshot dates (month starts) ending at obs_end month
end_month = obs_end.to_period("M").to_timestamp()   # first day of obs_end month
months = pd.date_range(end=end_month, periods=6, freq="MS")  # MS = month start
months = [m.date() for m in months]

print("obs_end:", obs_end.date())
print("Snapshot months:", months)

obs_end: 2017-10-17
Snapshot months: [datetime.date(2017, 5, 1), datetime.date(2017, 6, 1), datetime.date(2017, 7, 1), datetime.date(2017, 8, 1), datetime.date(2017, 9, 1), datetime.date(2017, 10, 1)]


In [5]:
import numpy as np

def assign_segment(recency_days, f30, m30):
    # same rule logic you used earlier
    if recency_days == 999:
        return "Never Purchased"
    if recency_days <= 30 and f30 >= 3 and m30 >= 300:
        return "Champion"
    if recency_days <= 30 and f30 >= 2:
        return "Loyal"
    if recency_days <= 30 and f30 >= 1:
        return "Recent"
    if recency_days > 90 and f30 >= 1:
        return "At Risk"
    return "Low Value"

all_snapshots = []

for month_start in months:
    month_start = pd.to_datetime(month_start)
    month_end = (month_start + pd.offsets.MonthEnd(0)).date()  # last day of that month

    # Latest available daily row per customer <= month_end
    snap_sql = f"""
    SELECT DISTINCT ON (customer_id)
        customer_id,
        feature_date,
        COALESCE(recency_days, 999) AS recency_days,
        COALESCE(f_30, 0) AS f_30,
        COALESCE(m_30, 0) AS m_30,
        COALESCE(views_30, 0) AS views_30,
        COALESCE(events_30, 0) AS events_30
    FROM hpce.customer_features_daily
    WHERE feature_date <= DATE '{month_end}'
    ORDER BY customer_id, feature_date DESC;
    """

    df = pd.read_sql(snap_sql, engine)
    df["snapshot_month"] = month_start.date()
    df["segment_label"] = [
        assign_segment(int(r), float(f), float(m))
        for r, f, m in zip(df["recency_days"], df["f_30"], df["m_30"])
    ]
    all_snapshots.append(df[["customer_id", "snapshot_month", "segment_label"]])

snapshots = pd.concat(all_snapshots, ignore_index=True)
print("Snapshots rows:", len(snapshots))
snapshots.head()


Snapshots rows: 441124


,customer_id,snapshot_month,segment_label
0,00012a2ce6f8dcda20d059ce98491703,2017-05-01,Never Purchased
1,000161a058600d5901f007fab4c27140,2017-05-01,Never Purchased
2,0001fd6190edaaf884bcaf3d49edf079,2017-05-01,Low Value
3,0004164d20a9e969af783496f3408652,2017-05-01,Recent
4,00046a560d407e99b969756e0b10f282,2017-05-01,Never Purchased


In [6]:
# Sort to compute previous month segment per customer
snapshots = snapshots.sort_values(["customer_id", "snapshot_month"]).reset_index(drop=True)

# prev segment per customer
snapshots["prev_segment_label"] = snapshots.groupby("customer_id")["segment_label"].shift(1)
snapshots["transitioned"] = (snapshots["prev_segment_label"].notna()) & (
    snapshots["prev_segment_label"] != snapshots["segment_label"]
)

# Insert into hpce.segment_transitions
out = snapshots[["customer_id", "snapshot_month", "prev_segment_label", "segment_label", "transitioned"]]

out.to_sql(
    name="segment_transitions",
    con=engine,
    schema="hpce",
    if_exists="append",
    index=False,
    chunksize=50_000,
    method="multi"
)

print("Inserted transitions:", len(out))

Inserted transitions: 441124
